In [23]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from scipy import signal, fftpack
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt

In [24]:
# Fungsi untuk menghitung highest peak PSD dari data
def calculate_highest_peak_psd(psd):
    return max(psd)

# List untuk menyimpan nilai highest peak PSD dan kelas data
highest_peak_psd_list = []
class_labels = []

# Daftar folder yang berisi data untuk masing-masing kelas
folder_paths = ['E:/LSTM/4_class/class1/', 
                'E:/LSTM/4_class/class2/', 
                'E:/LSTM/4_class/class3/', 
                'E:/LSTM/4_class/class4/']

In [25]:
# Loop untuk membaca dan memproses data dari folder
for class_idx, folder_path in enumerate(folder_paths):
    file_list = os.listdir(folder_path)
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        df_uncut = pd.read_csv(file_path)
        start_index = 1
        end_index = 1551
        df = df_uncut[start_index:end_index]
        data = df['Y'].values
        
        # Proses filtering, FFT, dan perhitungan PSD
        lowcut = 2.0
        highcut = 10.0
        fs = 5000000.0
        order = 5

        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(order, [low, high], btype='band')
    
        #Filter bandpass
        filtered_signal = signal.filtfilt(b, a, data)
    
        # One-sided FFT
        signal_fft = np.fft.fft(filtered_signal)
        signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
        n = len(filtered_signal)
        freq = signal_freq[0:int(n/2)]
        fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2
    
        # One-sided PSD
        psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
        freq_psd = signal_freq[0:int(n/2)]
        psd_one_side = psd[0:int(n/2)]
    
        # Hitung highest peak PSD dan tambahkan ke dalam list
        highest_peak_psd = calculate_highest_peak_psd(psd_one_side)
        highest_peak_psd_list.append(highest_peak_psd)
        
        # Tambahkan kelas (label) ke dalam list
        class_label = f'class{class_idx + 1}'
        class_labels.append(class_label)


In [26]:
# Buat DataFrame dari hasil ekstraksi fitur
data_dict = {'Highest_Peak_PSD': highest_peak_psd_list, 'Class_Label': class_labels}
df_features = pd.DataFrame(data_dict)

# Pisahkan fitur dan label untuk proses training dan testing
X = df_features['Highest_Peak_PSD'].values.reshape(-1, 1)
y = df_features['Class_Label'].values

# Bagi data menjadi data training dan data testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
print(class_labels)
print(len(class_labels))

['class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class1', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class2', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class3', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4', 'class4']

In [28]:
# Buat model Random Forest Classifier dan lakukan training
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [29]:
# Lakukan prediksi pada data testing
y_pred = rf_model.predict(X_test)

# Evaluasi model menggunakan metrik akurasi dan classification report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Accuracy: 30.0
Classification Report:
               precision    recall  f1-score   support

      class1       0.29      0.33      0.31         6
      class2       0.17      0.17      0.17         6
      class3       0.40      0.67      0.50         3
      class4       0.50      0.20      0.29         5

    accuracy                           0.30        20
   macro avg       0.34      0.34      0.32        20
weighted avg       0.32      0.30      0.29        20



In [30]:
# Simpan model ke file menggunakan joblib
model_filename = 'bone4.joblib'
joblib.dump(rf_model, model_filename)

['bone4.joblib']

In [31]:
# Mapping label kelas
class_mapping = {
    'class1': 'Density 30%',
    'class2': 'Density 50%',
    'class3': 'Density 70%',
    'class4': 'Density 90%'
}

# Bone Model Predictor

In [32]:
new_data_file = 'A50A3.csv'
df_new = pd.read_csv(new_data_file)
data_new = df_new['Y'].values

# Proses filtering, FFT, dan perhitungan PSD
lowcut = 2.0
highcut = 10.0
fs = 5000000.0
order = 5

nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = signal.butter(order, [low, high], btype='band')
    
#Filter bandpass
filtered_signal = signal.filtfilt(b, a, data)
    
# One-sided FFT
signal_fft = np.fft.fft(filtered_signal)
signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
n = len(filtered_signal)
freq = signal_freq[0:int(n/2)]
fft_amp = np.abs(signal_fft)[0:int(n/2)]/n*2
    
# One-sided PSD
psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
freq_psd = signal_freq[0:int(n/2)]
psd_one_side = psd[0:int(n/2)]
    
# Hitung highest peak PSD untuk data baru
highest_peak_psd_new = calculate_highest_peak_psd(psd_one_side)

# Bentuk data untuk prediksi
X_new = np.array([[highest_peak_psd_new]])


In [33]:
# Lakukan prediksi menggunakan model yang sudah disimpan
loaded_model = joblib.load(model_filename)
predicted_class_idx = loaded_model.predict(X_new)[0]
predicted_class = class_mapping.get(predicted_class_idx, 'Unknown')
print("Predicted Class:", predicted_class)

Predicted Class: Density 90%
